In [ ]:
# Install all required libraries
!pip install unbabel-comet
!pip install -q transformers accelerate evaluate bert_score rouge_score sacremoses nltk psutil pandas unbabel-comet

from huggingface_hub import login

# Paste your token here
login(token='hugging face token goes here')

# Download NLTK data needed for BLEU and ROUGE
import nltk
nltk.download('punkt')

# Imports
import time
import torch
import psutil
import logging
from nltk.translate.bleu_score import sentence_bleu
from transformers import MarianMTModel, MarianTokenizer
import evaluate

# Suppress unwanted info and warnings from Hugging Face and PyTorch Lightning
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

# Detect device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Helper to get MarianMT model name from model identifier
def get_model_name(model_identifier: str) -> str:
    return model_identifier

# Example sentences per language
paragraph_data = [
    {"code": "ar", "lang": "Arabic", "src": "أنا أتقدم للحصول على تأشيرة سياحية لزيارة الولايات المتحدة. أنوي البقاء لمدة أسبوعين وزيارة معالم مشهورة مثل تمثال الحرية وساحة التايمز. سأعود إلى بلدي بعد انتهاء عطلتي.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "es", "lang": "Spanish", "src": "Estoy solicitando una visa de turista para visitar los Estados Unidos. Tengo la intención de quedarme dos semanas y visitar lugares famosos como la Estatua de la Libertad y Times Square. Regresaré a mi país después de mis vacaciones.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "fr", "lang": "French", "src": "Je demande un visa de touriste pour visiter les États-Unis. J’ai l’intention de rester deux semaines et de visiter des sites célèbres comme la Statue de la Liberté et Times Square. Je retournerai dans mon pays après mes vacances.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "ru", "lang": "Russian", "src": "Я подаю заявку на туристическую визу для поездки в Соединенные Штаты. Я собираюсь остаться на две недели и посетить известные достопримечательности, такие как Статуя Свободы и Таймс-сквер. Я вернусь в свою страну после отпуска.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "ro", "lang": "Romanian", "src": "Solicit o viză turistică pentru a vizita Statele Unite. Intenționez să stau două săptămâni și să vizitez repere populare precum Statuia Libertății și Times Square. Mă voi întoarce în țara mea după vacanță.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "bg", "lang": "Bulgarian", "src": "Кандидатствам за туристическа виза за посещение на Съединените щати. Възнамерявам да остана две седмици и да посетя популярни забележителности като Статуята на свободата и Таймс Скуеър. Ще се върна в родината си след ваканцията.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "cs", "lang": "Czech", "src": "Žádám o turistické vízum pro návštěvu Spojených států. Mám v úmyslu zůstat dva týdny a navštívit známá místa jako Sochu svobody a Times Square. Po dovolené se vrátím do své domovské země.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "da", "lang": "Danish", "src": "Jeg ansøger om et turistvisum for at besøge USA. Jeg har til hensigt at blive i to uger og besøge kendte vartegn som Frihedsgudinden og Times Square. Jeg vender tilbage til mit hjemland efter ferien.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "de", "lang": "German", "src": "Ich beantrage ein Touristenvisum, um die Vereinigten Staaten zu besuchen. Ich beabsichtige, zwei Wochen zu bleiben und bekannte Sehenswürdigkeiten wie die Freiheitsstatue und den Times Square zu besuchen. Nach meinem Urlaub kehre ich in mein Heimatland zurück.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
    {"code": "et", "lang": "Estonian", "src": "Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.", "ref": "I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation."},
]

# Load evaluation metrics
bleu_mod     = evaluate.load("bleu")
bertscore_mod= evaluate.load("bertscore")
meteor_mod   = evaluate.load("meteor")
rouge_mod    = evaluate.load("rouge")
ter_mod      = evaluate.load("ter")
comet_mod    = evaluate.load("comet")

# Helpers to measure system resources
def measure_system():
    proc = psutil.Process()
    return proc.memory_info().rss / 1024**2, psutil.cpu_percent(interval=None)

# Translate a single text and measure speed
def translate_and_time(model, tokenizer, text: str):
    start = time.perf_counter()
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs)
    mid = time.perf_counter()
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    end = time.perf_counter()
    return pred, mid - start, end - start, len(tokenizer.tokenize(text))

# List of model identifiers (updated, moved Spanish before Arabic)
model_identifiers = [
    "Helsinki-NLP/opus-mt-ru-en", "Helsinki-NLP/opus-mt-tc-bible-big-roa-en", "Helsinki-NLP/opus-mt-ROMANCE-en",
    "Helsinki-NLP/opus-mt-az-en", "Helsinki-NLP/opus-mt-bat-en", "Helsinki-NLP/opus-mt-bg-en",
    "Helsinki-NLP/opus-mt-cel-en", "Helsinki-NLP/opus-mt-cs-en", "Helsinki-NLP/opus-mt-da-en",
    "Helsinki-NLP/opus-mt-de-en", "Helsinki-NLP/opus-mt-en_el_es_fi-en_el_es_fi", "Helsinki-NLP/opus-mt-et-en",
    "Helsinki-NLP/opus-mt-eu-en", "Helsinki-NLP/opus-mt-euq-en", "Helsinki-NLP/opus-mt-fi-en",
    "Helsinki-NLP/opus-mt-es-en", "Helsinki-NLP/opus-mt-ar-en",
    "Helsinki-NLP/opus-mt-fr-en"
]

# Run evaluation for each language and model
for item, model_identifier in zip(paragraph_data, model_identifiers):
    code, lang, src, ref = item["code"], item["lang"], item["src"], item["ref"]
    print(f"\n=== {lang} → English ===")

    # Load model & tokenizer
    tokenizer  = MarianTokenizer.from_pretrained(model_identifier)
    model      = MarianMTModel.from_pretrained(model_identifier).to(device)
    model.eval()

    # Measure before
    ram_before, cpu_before = measure_system()

    # Perform translation
    pred, t_first, t_total, token_count = translate_and_time(model, tokenizer, src)

    # Measure after
    ram_after, cpu_after = measure_system()

    # Compute metrics
    bleu_score   = bleu_mod.compute(predictions=[pred], references=[[ref]])["bleu"]
    bert_res     = bertscore_mod.compute(predictions=[pred], references=[ref], lang="en")
    meteor_score = meteor_mod.compute(predictions=[pred], references=[[ref]])["meteor"]
    rouge_score  = rouge_mod.compute(predictions=[pred], references=[ref])["rougeL"]
    ter_score    = ter_mod.compute(predictions=[pred], references=[[ref]])["score"]
    comet_res    = comet_mod.compute(sources=[src], predictions=[pred], references=[ref])
    comet_score  = next(v for v in comet_res.values() if isinstance(v, (int, float)))

    # Print results
    print(f"Source          : {src}")
    print(f"Reference       : {ref}")
    print(f"Prediction      : {pred}\n")

    print(f"--- Accuracy Metrics ---")
    print(f"BLEU            : {bleu_score:.4f}")
    print(f"BERTScore F1    : {bert_res['f1'][0]:.4f}")
    print(f"METEOR          : {meteor_score:.4f}")
    print(f"ROUGE-L         : {rouge_score:.4f}")
    print(f"TER             : {ter_score:.4f}")
    print(f"COMET           : {comet_score:.4f}\n")

    print(f"--- Speed Metrics ---")
    print(f"Time to 1st tok : {t_first:.4f} s")
    print(f"Total inf time  : {t_total:.4f} s")
    print(f"Tokens/sec      : {token_count / t_total:.2f}\n")

    print(f"--- Resource Usage ---")
    if device.type == "cuda":
        print(f"VRAM       : {(torch.cuda.max_memory_allocated() / 1024**2):.2f} MB")
    print(f"RAM        : {ram_after - ram_before:.2f} MB")
    print(f"CPU        : {cpu_after - cpu_before:.2f} %")
    print(f"Model size      : {(sum(p.numel() for p in model.parameters()) * 4 / 1024**2):.2f} MB")
    print("=" * 50)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


...

pytorch_model.bin:   0%|          | 0.00/298M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/298M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Source          : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.
Reference       : I am applying for a tourist visa to visit the United States. I intend to stay for two weeks and visit popular landmarks like the Statue of Liberty and Times Square. I will return to my home country after my vacation.
Prediction      : Taotlen turismiviisat Ameerika Ühendriikide külastamiseks. Kavatsen jääda kaheks nädalaks ja külastada kuulsaid vaatamisväärsusi nagu Vabadussammas ja Times Square. Pärast puhkust naasen oma koduriiki.

--- Accuracy Metrics ---
BLEU            : 0.0000
BERTScore F1    : 0.7625
METEOR          : 0.1057
ROUGE-L         : 0.0571
TER             : 95.1220
COMET           : 0.7336

--- Speed Metrics ---
Time to 1st tok : 6.4454 s
Total inf time  : 6.4472 s
Tokens/sec      : 14.11

--- Resource Usage ---
RAM        : -182.31 MB